[![AWS Data Wrangler](_static/logo.png "AWS Data Wrangler")](https://github.com/awslabs/aws-data-wrangler)

# 18 - QuickSight

For this tutorial we will use the public AWS COVID-19 data lake.

References:

* [A public data lake for analysis of COVID-19 data](https://aws.amazon.com/blogs/big-data/a-public-data-lake-for-analysis-of-covid-19-data/)
* [Exploring the public AWS COVID-19 data lake](https://aws.amazon.com/blogs/big-data/exploring-the-public-aws-covid-19-data-lake/)
* [CloudFormation template](https://covid19-lake.s3.us-east-2.amazonaws.com/cfn/CovidLakeStack.template.json)

*Please, install the Cloudformation template above to have access to the public data lake.*

*P.S. To be able to access the public data lake, you must allow explicitly QuickSight to access the related external bucket.*

In [1]:
import awswrangler as wr
from time import sleep

<strong>List users of QuickSight account<strong>

In [2]:
[{"username": user["UserName"], "role": user["Role"]} for user in wr.quicksight.list_users('default')]

[{'username': 'dev', 'role': 'ADMIN'}]

In [3]:
wr.catalog.databases()

,Database,Description
0,aws_data_wrangler,AWS Data Wrangler Test Arena - Glue Database
1,awswrangler_test,
2,covid-19,
3,default,Default Hive database


In [4]:
wr.catalog.tables(database="covid-19")

,Database,Table,Description,Columns,Partitions
0,covid-19,alleninstitute_comprehend_medical,Comprehend Medical results run against Allen I...,"paper_id, date, dx_name, test_name, procedure_...",
1,covid-19,alleninstitute_metadata,Metadata on papers pulled from the Allen Insti...,"cord_uid, sha, source_x, title, doi, pmcid, pu...",
2,covid-19,country_codes,Lookup table for country codes,"country, alpha-2 code, alpha-3 code, numeric c...",
3,covid-19,county_populations,Lookup table for population for each county ba...,"id, id2, county, state, population estimate 2018",
4,covid-19,covid_knowledge_graph_edges,AWS Knowledge Graph for COVID-19 data,"id, label, from, to, score",
5,covid-19,covid_knowledge_graph_nodes_author,AWS Knowledge Graph for COVID-19 data,"id, label, first, last, full_name",
6,covid-19,covid_knowledge_graph_nodes_concept,AWS Knowledge Graph for COVID-19 data,"id, label, entity, concept",
7,covid-19,covid_knowledge_graph_nodes_institution,AWS Knowledge Graph for COVID-19 data,"id, label, institution, country, settlement",
8,covid-19,covid_knowledge_graph_nodes_paper,AWS Knowledge Graph for COVID-19 data,"id, label, doi, sha_code, publish_time, source...",
9,covid-19,covid_knowledge_graph_nodes_topic,AWS Knowledge Graph for COVID-19 data,"id, label, topic, topic_num",


<strong>Create data source of QuickSight<strong>
Note: data source stores the connection information.

In [5]:
wr.quicksight.create_athena_data_source(
    name="covid-19",
    workgroup="primary",
    allowed_to_manage=["dev"]
)

In [6]:
wr.catalog.tables(database="covid-19", name_contains="nyt")

,Database,Table,Description,Columns,Partitions
0,covid-19,nytimes_counties,Data on COVID-19 cases from NY Times at US cou...,"date, county, state, fips, cases, deaths",
1,covid-19,nytimes_states,Data on COVID-19 cases from NY Times at US sta...,"date, state, fips, cases, deaths",


In [7]:
wr.athena.read_sql_query("SELECT * FROM nytimes_counties limit 10", database="covid-19", ctas_approach=False)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
5,2020-01-25,Orange,California,06059,1,0
6,2020-01-25,Cook,Illinois,17031,1,0
7,2020-01-25,Snohomish,Washington,53061,1,0
8,2020-01-26,Maricopa,Arizona,04013,1,0
9,2020-01-26,Los Angeles,California,06037,1,0


In [8]:
sql = """
SELECT 
  j.*, 
  co.Population, 
  co.county AS county2, 
  hb.* 
FROM 
  (
    SELECT 
      date, 
      county, 
      state, 
      fips, 
      cases as confirmed, 
      deaths 
    FROM "covid-19".nytimes_counties
  ) j 
  LEFT OUTER JOIN (
    SELECT 
      DISTINCT county, 
      state, 
      "population estimate 2018" AS Population 
    FROM 
      "covid-19".county_populations 
    WHERE 
      state IN (
        SELECT 
          DISTINCT state 
        FROM 
          "covid-19".nytimes_counties
      ) 
      AND county IN (
        SELECT 
          DISTINCT county as county 
        FROM "covid-19".nytimes_counties
      )
  ) co ON co.county = j.county 
  AND co.state = j.state 
  LEFT OUTER JOIN (
    SELECT 
      count(objectid) as Hospital, 
      fips as hospital_fips, 
      sum(num_licensed_beds) as licensed_beds, 
      sum(num_staffed_beds) as staffed_beds, 
      sum(num_icu_beds) as icu_beds, 
      avg(bed_utilization) as bed_utilization, 
      sum(
        potential_increase_in_bed_capac
      ) as potential_increase_bed_capacity 
    FROM "covid-19".hospital_beds 
    WHERE 
      fips in (
        SELECT 
          DISTINCT fips 
        FROM 
          "covid-19".nytimes_counties
      ) 
    GROUP BY 
      2
  ) hb ON hb.hospital_fips = j.fips
"""

wr.athena.read_sql_query(sql, database="covid-19", ctas_approach=False)

,date,county,state,fips,confirmed,deaths,population,county2,Hospital,hospital_fips,licensed_beds,staffed_beds,icu_beds,bed_utilization,potential_increase_bed_capacity
0,2020-04-12,Park,Montana,30067,7,0,16736,Park,0,30067,25,25,4,0.432548,0
1,2020-04-12,Ravalli,Montana,30081,3,0,43172,Ravalli,0,30081,25,25,5,0.567781,0
2,2020-04-12,Silver Bow,Montana,30093,11,0,34993,Silver Bow,0,30093,98,71,11,0.551457,27
3,2020-04-12,Clay,Nebraska,31035,2,0,6214,Clay,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
4,2020-04-12,Cuming,Nebraska,31039,2,0,8940,Cuming,0,31039,25,25,4,0.204493,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227684,2020-06-11,Hockley,Texas,48219,28,1,22980,Hockley,0,48219,48,48,8,0.120605,0
227685,2020-06-11,Hudspeth,Texas,48229,11,0,4795,Hudspeth,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
227686,2020-06-11,Jones,Texas,48253,633,0,19817,Jones,0,48253,45,7,1,0.718591,38
227687,2020-06-11,La Salle,Texas,48283,4,0,7531,La Salle,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>


<strong>Create Dataset with custom SQL option<strong>

In [9]:
wr.quicksight.create_athena_dataset(
    name="covid19-nytimes-usa",
    sql=sql,
    sql_name='CustomSQL',
    data_source_name="covid-19",
    import_mode='SPICE',
    allowed_to_manage=["dev"]
)

In [10]:
ingestion_id = wr.quicksight.create_ingestion("covid19-nytimes-usa")

<strong>Wait ingestion<strong>

In [11]:
while wr.quicksight.describe_ingestion(ingestion_id=ingestion_id, dataset_name="covid19-nytimes-usa")["IngestionStatus"] not in ["COMPLETED", "FAILED"]:
    sleep(1)

<strong>Describe last ingestion<strong>

In [12]:
wr.quicksight.describe_ingestion(ingestion_id=ingestion_id, dataset_name="covid19-nytimes-usa")["RowInfo"]

{'RowsIngested': 227689, 'RowsDropped': 0}

<strong>List all ingestions<strong>

In [13]:
[{"time": user["CreatedTime"], "source": user["RequestSource"]} for user in wr.quicksight.list_ingestions("covid19-nytimes-usa")]

[{'time': datetime.datetime(2020, 6, 12, 15, 13, 46, 996000, tzinfo=tzlocal()),
  'source': 'MANUAL'},
 {'time': datetime.datetime(2020, 6, 12, 15, 13, 42, 344000, tzinfo=tzlocal()),
  'source': 'MANUAL'}]

<strong>Create new dataset from a table directly<strong>

In [14]:
wr.quicksight.create_athena_dataset(
    name="covid-19-tableau_jhu",
    table="tableau_jhu",
    data_source_name="covid-19",
    database="covid-19",
    import_mode='DIRECT_QUERY',
    rename_columns={
        "cases": "Count_of_Cases", 
        "combined_key": "County"
    },
    cast_columns_types={
        "Count_of_Cases": "INTEGER"
    },
    tag_columns={
        "combined_key": [{"ColumnGeographicRole": "COUNTY"}]
    },
    allowed_to_manage=["dev"]
)

<strong>Cleaning up<strong>

In [15]:
wr.quicksight.delete_data_source("covid-19")
wr.quicksight.delete_dataset("covid19-nytimes-usa")
wr.quicksight.delete_dataset("covid-19-tableau_jhu")